<a href="https://colab.research.google.com/github/dvsmihir1/ML-Lab/blob/main/ML_lab_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas
import numpy